## Insurance Case Study
### Steps
1. Filter out the designed State
2. Drop rows based on conditions
3. Merge 3 datasets
4. Calculate top 10 market share insurance in each state
5. Report the rate for every contract offered by the top-10 biggest market share insurance companies in each state
6. Report the weighted average of the UOD rate of every contract offered by the top-10 biggest market share insurance companies in each state



In [1]:
import pandas as pd
import numpy as np

In [2]:
enroll_info = pd.read_csv("CPSC_Enrollment_Info_2022_01.csv")
monthly_report = pd.read_excel("Monthly_Report_By_Plan_2022_01.xlsx",header=5)
major_orgs = pd.read_excel("MajorInsuranceOrgs.xlsx")

### 1.Filter out the designed State 

In [3]:
State_list = ['HI','MI','MN','MS','NY','OK','SD','TN']
enroll_info = enroll_info[enroll_info['State'].isin(State_list)]

### 2.Drop rows based on conditions
1. exclude all "Exxx" contracts
2. drop rows where value in enrollment column is missing or mared with *

In [4]:
enroll_info = enroll_info[enroll_info["Contract Number"].str[0] != 'S']
enroll_info = enroll_info.dropna(subset=['Enrollment'])
enroll_info = enroll_info[enroll_info.Enrollment != '*']
enroll_info_f = enroll_info[['Contract Number','State','Enrollment']]

In [5]:
enroll_info_f.head()

,Contract Number,State,Enrollment
1255,E0654,MI,19
1268,E0654,MI,11
1271,E0654,MI,15
1277,E0654,MI,17
1280,E0654,MI,63


In [6]:
enroll_info_f["Enrollment"] = pd.to_numeric(enroll_info_f["Enrollment"])

/Users/rosalind/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
enroll_info_ff = enroll_info_f.groupby(["State","Contract Number"])["Enrollment"].sum().reset_index(name ='total_enrollees')

In [8]:
enroll_info_ff.head()

,State,Contract Number,total_enrollees
0,HI,H0028,8596
1,HI,H1230,34248
2,HI,H1924,32
3,HI,H2001,1032
4,HI,H2228,35386


### 3.Merge dataset to create the final dataset

In [9]:
monthly_report_f = monthly_report[['Contract Number','Organization Marketing Name']]
left_join_1 = pd.merge(enroll_info_f, monthly_report_f, on='Contract Number', how='left')

In [10]:
left_join_2 = pd.merge(left_join_1, major_orgs, on='Organization Marketing Name', how='left')

In [11]:
df = left_join_2

In [12]:
df.head()

,Contract Number,State,Enrollment,Organization Marketing Name,MajorInsuranceOrgName
0,E0654,MI,19,TEAMStar Medicare Part D Prescription Drug Pro...,NaN
1,E0654,MI,11,TEAMStar Medicare Part D Prescription Drug Pro...,NaN
2,E0654,MI,15,TEAMStar Medicare Part D Prescription Drug Pro...,NaN
3,E0654,MI,17,TEAMStar Medicare Part D Prescription Drug Pro...,NaN
4,E0654,MI,63,TEAMStar Medicare Part D Prescription Drug Pro...,NaN


### 4.top-10 biggest market share insurance companies in each state

In [14]:
# calculate the total number of enrollees of each company in each state
df["Enrollment"] = pd.to_numeric(df["Enrollment"])
company_enroll = df.groupby(['State','MajorInsuranceOrgName'])['Enrollment'].sum().reset_index(name ='num_enrollees')
company_enroll

,State,MajorInsuranceOrgName,num_enrollees
0,HI,Humana,352436
1,HI,Kaiser,342480
2,HI,Lasso Healthcare,64
3,HI,UnitedHealthcare,3569262
4,MI,Aetna Health Inc.,147290
5,MI,AmeriHealth Caritas VIP Care Plus,2920
6,MI,HAP Senior Plus,773080
7,MI,HAP Senior Plus (PPO),56490
8,MI,LIFECIRCLES,692
9,MI,PACE Central Michigan,190


In [15]:
# calculate the total number of enrollees in the state
state_enroll = df.groupby(['State'])['Enrollment'].sum().reset_index(name ='total_enrollees')
state_enroll

,State,total_enrollees
0,HI,4422876
1,MI,3590092
2,MN,3545047
3,MS,3922269
4,NY,6052770
5,OK,1580541
6,SD,1079268
7,TN,2145962


In [16]:
#calculate the market share of each company in each state
company_enroll = pd.merge(company_enroll, state_enroll, on='State', how='inner')
company_enroll['market_share'] = company_enroll['num_enrollees'].div(company_enroll['total_enrollees'])
df_top10 = company_enroll.groupby(['State']).apply(lambda x: x.sort_values(["market_share"],ascending = False).reset_index(drop=True))
company_enroll

,State,MajorInsuranceOrgName,num_enrollees,total_enrollees,market_share
0,HI,Humana,352436,4422876,0.079685
1,HI,Kaiser,342480,4422876,0.077434
2,HI,Lasso Healthcare,64,4422876,0.000014
3,HI,UnitedHealthcare,3569262,4422876,0.807000
4,MI,Aetna Health Inc.,147290,3590092,0.041027
5,MI,AmeriHealth Caritas VIP Care Plus,2920,3590092,0.000813
6,MI,HAP Senior Plus,773080,3590092,0.215337
7,MI,HAP Senior Plus (PPO),56490,3590092,0.015735
8,MI,LIFECIRCLES,692,3590092,0.000193
9,MI,PACE Central Michigan,190,3590092,0.000053


##### Based on the the table above, we can figure out the top10 biggest market share insurance company in each state. And I also drop those companies ranked after 10

In [17]:
df_top10 = company_enroll.drop(index=[8,9,11,14])
df_top10 = df_top10.groupby(['State']).apply(lambda x: x.sort_values(["market_share"],ascending = False).reset_index(drop=True))

In [18]:
#index_names = company_enroll[(company_enroll['State'] == 'MI') & ((df['MajorInsuranceOrgName'] == 'LIFECIRCLES')|(df['MajorInsuranceOrgName'] == 'PACE of Southwest Michigan')(df['MajorInsuranceOrgName'] == 'Thome PACE')|(df['MajorInsuranceOrgName'] == 'PACE Central Michigan'))].index
#df_top10 = df_top10.drop(index_names, inplace = True)

In [35]:
df_top10_docs = df_top10.iloc[:, [1,4]]
df_top10_docs.style.format({
    'market_share': '{:,.2%}'.format
})

In [19]:
df_top10.style.format({
    'market_share': '{:,.4%}'.format
})

In [23]:
df_top10.to_excel("Top 10 biggest mkt share company.xlsx")


### 5.Report the rate for every contract offered by the top-10 biggest market share insurance companies in each state

#### HI

In [36]:
states_HI = ['UnitedHealthcare','Humana','Kaiser','Lasso Healthcare']
df_HI = df[df['MajorInsuranceOrgName'].isin(states_HI)]
df_HI_1 = df_HI.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_HI_1['rate_contract'] = (df_HI_1['num_contrct']/len(df_HI.index)).round(6)
df_HI_1['State'] = 'HI'
df_HI_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Humana,H0028,369,0.4857%,HI
1,Humana,H1036,2093,2.7550%,HI
2,Humana,H2944,39,0.0513%,HI
3,Kaiser,H1230,150,0.1974%,HI
4,Lasso Healthcare,H1924,24,0.0316%,HI
5,UnitedHealthcare,H0251,354,0.4660%,HI
6,UnitedHealthcare,H0271,1650,2.1719%,HI
7,UnitedHealthcare,H0294,450,0.5923%,HI
8,UnitedHealthcare,H0710,603,0.7937%,HI
9,UnitedHealthcare,H1278,294,0.3870%,HI


In [43]:
df_HI_1.to_excel("HI.xlsx")

#### MI

In [37]:
states_MI = ['Priority Health Medicare','UnitedHealthcare','HAP Senior Plus','Aetna Health Inc.',
             'HAP Senior Plus (PPO)','Upper Peninsula Health Plan (UPHP) MI Health Link',
            'AmeriHealth Caritas VIP Care Plus','PACE Southeast Michigan','Upper Peninsula Health Plan',
            'Senior Care Partners P.A.C.E.']
df_MI = df[df['MajorInsuranceOrgName'].isin(states_MI)]
df_MI_1 = df_MI.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_MI_1['rate_contract'] = (df_MI_1['num_contrct']/len(df_MI.index)).round(6)
df_MI_1['State'] = 'MI'
df_MI_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.1541%,MI
1,Aetna Health Inc.,H2663,84,0.0989%,MI
2,Aetna Health Inc.,H3146,320,0.3769%,MI
3,Aetna Health Inc.,H3192,297,0.3498%,MI
4,Aetna Health Inc.,H3219,560,0.6595%,MI
5,Aetna Health Inc.,H3239,105,0.1237%,MI
6,Aetna Health Inc.,H3288,3339,3.9323%,MI
7,Aetna Health Inc.,H3312,1265,1.4898%,MI
8,AmeriHealth Caritas VIP Care Plus,H0192,3,0.0035%,MI
9,HAP Senior Plus,H2354,1408,1.6582%,MI


In [42]:
df_MI_1.to_excel("MI.xlsx")

#### MN

In [38]:
states_MN = ['UCare','UnitedHealthcare','BlueCrossBlueShield','Aetna Health Inc.',
             'HealthPartners',"UCare's MSHO",'PrimeWest Health',
             'South Country Health Alliance','Itasca Medical Care/IMCare Classic',
            'Lasso Healthcare']
df_MN = df[df['MajorInsuranceOrgName'].isin(states_MN)]
df_MN_1 = df_MN.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_MN_1['rate_contract'] = (df_MN_1['num_contrct']/len(df_MN.index)).round(6)
df_MN_1['State'] = 'MN'
df_MN_1.style.format({
    'rate_contract': '{:,.4%}'.format
})



,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.1272%,MN
1,Aetna Health Inc.,H2663,84,0.0966%,MN
2,Aetna Health Inc.,H3146,320,0.3681%,MN
3,Aetna Health Inc.,H3192,297,0.3416%,MN
4,Aetna Health Inc.,H3219,560,0.6441%,MN
5,Aetna Health Inc.,H3239,105,0.1208%,MN
6,Aetna Health Inc.,H3288,3339,3.8405%,MN
7,Aetna Health Inc.,H3312,1265,1.4550%,MN
8,BlueCrossBlueShield,H0107,66,0.0759%,MN
9,BlueCrossBlueShield,H1732,186,0.2139%,MN


In [39]:
df_MN_1.to_excel("MN.xlsx")

#### MS

In [44]:
states_MS = ['Humana','UnitedHealthcare','Aetna Health Inc.','Lasso Healthcare']
df_MS = df[df['MajorInsuranceOrgName'].isin(states_MS)]
df_MS_1 = df_MS.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_MS_1['rate_contract'] = (df_MS_1['num_contrct']/len(df_MS.index)).round(6)
df_MS_1['State'] = 'MN'
df_MS_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.1840%,MN
1,Aetna Health Inc.,H2663,84,0.1015%,MN
2,Aetna Health Inc.,H3146,320,0.3866%,MN
3,Aetna Health Inc.,H3192,297,0.3588%,MN
4,Aetna Health Inc.,H3219,560,0.6766%,MN
5,Aetna Health Inc.,H3239,105,0.1269%,MN
6,Aetna Health Inc.,H3288,3339,4.0341%,MN
7,Aetna Health Inc.,H3312,1265,1.5283%,MN
8,Humana,H0028,369,0.4458%,MN
9,Humana,H1036,2093,2.5287%,MN


In [45]:
df_MS_1.to_excel("MS.xlsx")

#### NY

In [46]:
states_NY = ['UnitedHealthcare','Aetna Health Inc.','MVP HEALTH CARE','BlueCrossBlueShield',
            'MetroPlus Health Plan','VillageCareMAX','Bright Health','ArchCare Advantage',
            'Healthfirst Medicare Plan','Catholic Health LIFE']
df_NY = df[df['MajorInsuranceOrgName'].isin(states_NY)]
df_NY_1 = df_NY.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_NY_1['rate_contract'] = (df_NY_1['num_contrct']/len(df_NY.index)).round(6)
df_NY_1['State'] = 'NY'
df_NY_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.1729%,NY
1,Aetna Health Inc.,H2663,84,0.1005%,NY
2,Aetna Health Inc.,H3146,320,0.3830%,NY
3,Aetna Health Inc.,H3192,297,0.3554%,NY
4,Aetna Health Inc.,H3219,560,0.6702%,NY
5,Aetna Health Inc.,H3239,105,0.1257%,NY
6,Aetna Health Inc.,H3288,3339,3.9961%,NY
7,Aetna Health Inc.,H3312,1265,1.5139%,NY
8,ArchCare Advantage,H1777,10,0.0120%,NY
9,BlueCrossBlueShield,H0107,66,0.0790%,NY


In [47]:
df_NY_1.to_excel("NY.xlsx")

#### OK

In [48]:
states_OK = ['UnitedHealthcare','Aetna Health Inc.','BlueCrossBlueShield','Humana']
df_OK = df[df['MajorInsuranceOrgName'].isin(states_OK)]
df_OK_1 = df_OK.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_OK_1['rate_contract'] = (df_OK_1['num_contrct']/len(df_OK.index)).round(6)
df_OK_1['State'] = 'OK'
df_OK_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.1668%,OK
1,Aetna Health Inc.,H2663,84,0.1000%,OK
2,Aetna Health Inc.,H3146,320,0.3810%,OK
3,Aetna Health Inc.,H3192,297,0.3536%,OK
4,Aetna Health Inc.,H3219,560,0.6667%,OK
5,Aetna Health Inc.,H3239,105,0.1250%,OK
6,Aetna Health Inc.,H3288,3339,3.9754%,OK
7,Aetna Health Inc.,H3312,1265,1.5061%,OK
8,BlueCrossBlueShield,H0107,66,0.0786%,OK
9,BlueCrossBlueShield,H1732,186,0.2214%,OK


In [49]:
df_OK_1.to_excel("OK.xlsx")

#### SD

In [50]:
states_SD = ['UnitedHealthcare','Aetna Health Inc.','Humana','HealthPartners',
             'Lasso Healthcare']
df_SD = df[df['MajorInsuranceOrgName'].isin(states_SD)]
df_SD_1 = df_SD.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_SD_1['rate_contract'] = (df_SD_1['num_contrct']/len(df_SD.index)).round(6)
df_SD_1['State'] = 'SD'
df_SD_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.1781%,SD
1,Aetna Health Inc.,H2663,84,0.1010%,SD
2,Aetna Health Inc.,H3146,320,0.3847%,SD
3,Aetna Health Inc.,H3192,297,0.3570%,SD
4,Aetna Health Inc.,H3219,560,0.6732%,SD
5,Aetna Health Inc.,H3239,105,0.1262%,SD
6,Aetna Health Inc.,H3288,3339,4.0139%,SD
7,Aetna Health Inc.,H3312,1265,1.5207%,SD
8,HealthPartners,H2422,12,0.0144%,SD
9,HealthPartners,H2462,403,0.4845%,SD


In [51]:
df_SD_1.to_excel("SD.xlsx")

#### TN

In [52]:
states_TN = ['UnitedHealthcare','BlueCrossBlueShield','Aetna Health Inc.']
df_TN = df[df['MajorInsuranceOrgName'].isin(states_TN)]
df_TN_1 = df_TN.groupby(['MajorInsuranceOrgName','Contract Number']).size().reset_index(name ='num_contrct')
df_TN_1['rate_contract'] = (df_TN_1['num_contrct']/len(df_TN.index)).round(6)
df_TN_1['State'] = 'TN'
df_TN_1.style.format({
    'rate_contract': '{:,.4%}'.format
})

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State
0,Aetna Health Inc.,H1608,980,1.2026%,TN
1,Aetna Health Inc.,H2663,84,0.1031%,TN
2,Aetna Health Inc.,H3146,320,0.3927%,TN
3,Aetna Health Inc.,H3192,297,0.3645%,TN
4,Aetna Health Inc.,H3219,560,0.6872%,TN
5,Aetna Health Inc.,H3239,105,0.1288%,TN
6,Aetna Health Inc.,H3288,3339,4.0974%,TN
7,Aetna Health Inc.,H3312,1265,1.5523%,TN
8,BlueCrossBlueShield,H0107,66,0.0810%,TN
9,BlueCrossBlueShield,H1732,186,0.2282%,TN


In [53]:
df_TN_1.to_excel("TN.xlsx")

### 6.Report the weighted average of the UOD rate in each state

In excel, I filter the MeasureCode = HDO, drop missing rate,drop unuseful columns, and change the column name 'CMSContractNumber' to 'Contract Number' so that we can merge dataset easily, finally I got 'UOD.xlsx'

In [54]:
df_UOD = pd.read_excel("UOD.xlsx")

#### HI

In [118]:
df_HI_2= pd.merge(df_HI_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_HI = pd.merge(df_HI_2, df_UOD, on='Contract Number', how='left')
df_UOD_HI = df_UOD_HI.dropna()

In [119]:
df_UOD_HI

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State,total_enrollees,UOD Rate
0,Humana,H0028,369,0.004857,HI,8596.0,5.21
3,Kaiser,H1230,150,0.001974,HI,34248.0,3.12
11,UnitedHealthcare,H2001,66048,0.869396,HI,1032.0,4.73
13,UnitedHealthcare,H2228,1089,0.014335,HI,35386.0,6.68


In [120]:
df_UOD_HI['mul'] = df_UOD_HI['UOD Rate'].mul(df_UOD_HI['total_enrollees'])

In [121]:
df_UOD_HI

,MajorInsuranceOrgName,Contract Number,num_contrct,rate_contract,State,total_enrollees,UOD Rate,mul
0,Humana,H0028,369,0.004857,HI,8596.0,5.21,44785.16
3,Kaiser,H1230,150,0.001974,HI,34248.0,3.12,106853.76
11,UnitedHealthcare,H2001,66048,0.869396,HI,1032.0,4.73,4881.36
13,UnitedHealthcare,H2228,1089,0.014335,HI,35386.0,6.68,236378.48


In [122]:
a = df_UOD_HI.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')

In [123]:
a

,MajorInsuranceOrgName,sum
0,Humana,44785.16
1,Kaiser,106853.76
2,UnitedHealthcare,241259.84


In [124]:
b = df_UOD_HI.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')

In [125]:
b

,MajorInsuranceOrgName,sum_enroll
0,Humana,8596.0
1,Kaiser,34248.0
2,UnitedHealthcare,36418.0


In [126]:
df_avgUOD_HI = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')

In [127]:
df_avgUOD_HI

,MajorInsuranceOrgName,sum,sum_enroll
0,Humana,44785.16,8596.0
1,Kaiser,106853.76,34248.0
2,UnitedHealthcare,241259.84,36418.0


In [128]:
df_avgUOD_HI['Weighted Avg UOD Rate'] = df_avgUOD_HI["sum"].div(df_avgUOD_HI["sum_enroll"])

In [129]:
df_avgUOD_HI.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)

In [130]:
df_avgUOD_HI

,MajorInsuranceOrgName,sum,sum_enroll,Weighted Avg UOD Rate
2,UnitedHealthcare,241259.84,36418.0,6.624742
0,Humana,44785.16,8596.0,5.210000
1,Kaiser,106853.76,34248.0,3.120000


In [131]:
df_avgUOD_HI.to_excel("HI.xlsx")

In [59]:
#df = df.reindex(columns=['Available_Since_Date','Product_name', 'Unit_Price','No_Of_Units','Available_Quantity'])

#### MI

In [132]:
df_MI_2= pd.merge(df_MI_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_MI = pd.merge(df_MI_2, df_UOD, on='Contract Number', how='left')
df_UOD_MI = df_UOD_MI.dropna()
df_UOD_MI['mul'] = df_UOD_MI['UOD Rate'].mul(df_UOD_MI['total_enrollees'])
a = df_UOD_MI.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_MI.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_MI = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_MI['Weighted Avg UOD Rate'] = df_avgUOD_MI["sum"].div(df_avgUOD_MI["sum_enroll"])
df_avgUOD_MI.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_MI
df_avgUOD_MI.to_excel("MI.xlsx")

In [133]:
df_avgUOD_MI

,MajorInsuranceOrgName,sum,sum_enroll,Weighted Avg UOD Rate
2,HAP Senior Plus (PPO),76073.20,9415.0,8.080000
4,UnitedHealthcare,204657.75,36055.0,5.676265
3,Priority Health Medicare,707719.60,133532.0,5.300000
0,Aetna Health Inc.,53961.70,13390.0,4.030000
1,HAP Senior Plus,281822.80,70280.0,4.010000
5,Upper Peninsula Health Plan,1976.76,612.0,3.230000


#### MN

In [134]:
df_MN_2= pd.merge(df_MN_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_MN = pd.merge(df_MN_2, df_UOD, on='Contract Number', how='left')
df_UOD_MN = df_UOD_MN.dropna()
df_UOD_MN['mul'] = df_UOD_MN['UOD Rate'].mul(df_UOD_MN['total_enrollees'])
a = df_UOD_MN.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_MN.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_MN = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_MN['Weighted Avg UOD Rate'] = df_avgUOD_MN["sum"].div(df_avgUOD_MN["sum_enroll"])
df_avgUOD_MN.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_MN
df_avgUOD_MN.to_excel("MN.xlsx")

In [135]:
df_avgUOD_MN

,MajorInsuranceOrgName,sum,sum_enroll,Weighted Avg UOD Rate
2,HealthPartners,43935.28,5924.0,7.416489
3,PrimeWest Health,13763.70,1865.0,7.380000
4,South Country Health Alliance,7736.44,1438.0,5.380000
6,UCare's MSHO,77296.56,14421.0,5.360000
0,Aetna Health Inc.,88254.18,16527.0,5.340000
7,UnitedHealthcare,99618.59,21091.0,4.723275
5,UCare,503280.57,113506.0,4.433956
1,BlueCrossBlueShield,216842.71,53547.0,4.049577


#### MS

In [136]:
df_MS_2= pd.merge(df_MS_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_MS = pd.merge(df_MS_2, df_UOD, on='Contract Number', how='left')
df_UOD_MS = df_UOD_MS.dropna()
df_UOD_MS['mul'] = df_UOD_MS['UOD Rate'].mul(df_UOD_MS['total_enrollees'])
a = df_UOD_MS.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_MS.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_MS = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_MS['Weighted Avg UOD Rate'] = df_avgUOD_MS["sum"].div(df_avgUOD_MS["sum_enroll"])
df_avgUOD_MS.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_MS
df_avgUOD_MS.to_excel("MS.xlsx")

#### NY

In [137]:
df_NY_2= pd.merge(df_NY_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_NY = pd.merge(df_NY_2, df_UOD, on='Contract Number', how='left')
df_UOD_NY = df_UOD_NY.dropna()
df_UOD_NY['mul'] = df_UOD_NY['UOD Rate'].mul(df_UOD_NY['total_enrollees'])
a = df_UOD_NY.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_NY.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_NY = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_NY['Weighted Avg UOD Rate'] = df_avgUOD_NY["sum"].div(df_avgUOD_NY["sum_enroll"])
df_avgUOD_NY.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_NY
df_avgUOD_NY.to_excel("NY.xlsx")

#### OK

In [138]:
df_OK_2= pd.merge(df_OK_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_OK = pd.merge(df_OK_2, df_UOD, on='Contract Number', how='left')
df_UOD_OK = df_UOD_OK.dropna()
df_UOD_OK['mul'] = df_UOD_OK['UOD Rate'].mul(df_UOD_OK['total_enrollees'])
a = df_UOD_OK.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_OK.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_OK = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_OK['Weighted Avg UOD Rate'] = df_avgUOD_OK["sum"].div(df_avgUOD_OK["sum_enroll"])
df_avgUOD_OK.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_OK
df_avgUOD_OK.to_excel("OK.xlsx")

#### SD

In [139]:
df_SD_2= pd.merge(df_SD_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_SD = pd.merge(df_SD_2, df_UOD, on='Contract Number', how='left')
df_UOD_SD = df_UOD_SD.dropna()
df_UOD_SD['mul'] = df_UOD_SD['UOD Rate'].mul(df_UOD_SD['total_enrollees'])
a = df_UOD_SD.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_SD.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_SD = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_SD['Weighted Avg UOD Rate'] = df_avgUOD_SD["sum"].div(df_avgUOD_SD["sum_enroll"])
df_avgUOD_SD.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_SD
df_avgUOD_SD.to_excel("SD.xlsx")

#### TN

In [140]:
df_TN_2= pd.merge(df_TN_1, enroll_info_ff, on=["State","Contract Number"], how = 'left')
df_UOD_TN = pd.merge(df_TN_2, df_UOD, on='Contract Number', how='left')
df_UOD_TN = df_UOD_TN.dropna()
df_UOD_TN['mul'] = df_UOD_TN['UOD Rate'].mul(df_UOD_TN['total_enrollees'])
a = df_UOD_TN.groupby(["MajorInsuranceOrgName"])["mul"].sum().reset_index(name ='sum')
b = df_UOD_TN.groupby(["MajorInsuranceOrgName"])["total_enrollees"].sum().reset_index(name ='sum_enroll')
df_avgUOD_TN = pd.merge(a, b, on='MajorInsuranceOrgName', how='left')
df_avgUOD_TN['Weighted Avg UOD Rate'] = df_avgUOD_TN["sum"].div(df_avgUOD_TN["sum_enroll"])
df_avgUOD_TN.sort_values(by="Weighted Avg UOD Rate",ascending=False,inplace=True)
df_avgUOD_TN
df_avgUOD_TN.to_excel("TN.xlsx")